Aligning and shifting data to a common baseline for the year 2000

In [ ]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import xarray as xr
from scipy.interpolate import interp1d

# Apply Style Rule
plt.style.use("seaborn-v0_8-poster")

# Ensure processed directory exists
os.makedirs("../data/processed", exist_ok=True)

# ## 1. WLS quadratic fit
# Purpose: calculate rate and acceleration using `sm.WLS`.
# Model: y = a + b(t-tc) + 0.5*c(t-tc)^2
# where b = rate and c = acceleration.

def calculate_kinematics_sm(time, val, sigma, window_years=30.):
    """
    Calculates rate and acceleration with uncertainties
    using a moving window regression. 
    
    Parameters:
    -----------
    time : array-like
        Time vector (usually in decimal years).
    val : array-like
        Measured values (GMST or GMSLR).
    sigma : array-like
        1-sigma uncertainties for each data point.
    span_years : float
        The bandwidth/window size in years (default 30)
        
    Returns:
    --------
    pd.DataFrame containing rates, accelerations, and their standard errors.
    """
    win_size = int(window_years * 12)
    half_win = win_size // 2
    
    n = len(time)
    results = {
        'rate': np.full(n, np.nan), 'rate_se': np.full(n, np.nan),
        'accel': np.full(n, np.nan), 'accel_se': np.full(n, np.nan)
    }
    
    for i in range(half_win, n - half_win):
        t_w = time[i - half_win : i + half_win]
        y_w = val[i - half_win : i + half_win]
        s_w = sigma[i - half_win : i + half_win]
        
        # Center time for numerical stability
        tc = np.mean(t_w)
        dt = t_w - tc
        
        # Design matrix: Constant, Time (Rate), 0.5 * Time^2 (Acceleration)
        X = np.column_stack([np.ones(len(dt)), dt, 0.5 * dt**2])
        weights = 1.0 / (s_w**2)
        
        # statsmodels WLS operation
        res = sm.WLS(y_w, X, weights=weights).fit()
        
        results['rate'][i] = model.params[1]
        results['rate_se'][i] = model.bse[1]
        results['accel'][i] = model.params[2]
        results['accel_se'][i] = model.bse[2]
        
    return pd.DataFrame(results, index=time)

def calculate_kernel_kinematics_sm(time, val, sigma, span_years=20.):
    """
    Estimates Rate and Acceleration across the full time series, including edges,
    using Kernel-Weighted Local Polynomial Regression via statsmodels.
    
    Parameters:
    -----------
    time : array-like
        Time vector (usually in decimal years).
    val : array-like
        Measured values (GMST or GMSLR).
    sigma : array-like
        1-sigma uncertainties for each data point.
    span_years : float
        The bandwidth/window size in years (default 20)
        
    Returns:
    --------
    pd.DataFrame containing rates, accelerations, and their standard errors.
    """
    n = len(time)
    results = {
        'rate': np.full(n, np.nan), 'rate_se': np.full(n, np.nan),
        'accel': np.full(n, np.nan), 'accel_se': np.full(n, np.nan)
    }
    
    h = span_years 

    for i in range(n):
        t0 = time[i]
        d = np.abs(time - t0)
        u = d / h
        
        # Tricube kernel: (1 - u^3)^3
        kernel_weights = np.where(u <= 1, (1 - u**3)**3, 0)
        
        # Rule: Weight = Kernel / sigma^2
        combined_weights = kernel_weights / (sigma**2)
        
        mask = combined_weights > 1e-12 
        if np.sum(mask) < 12: # Sufficient degrees of freedom for quadratic fit
            continue
            
        tw, yw, ww = time[mask], val[mask], combined_weights[mask]
        dt = tw - t0
        X = np.column_stack([np.ones(len(dt)), dt, 0.5 * dt**2])
        
        # statsmodels WLS engine
        try:
            model = sm.WLS(yw, X, weights=ww).fit()
            results['rate'][i] = model.params[1]
            results['rate_se'][i] = model.bse[1]
            results['accel'][i] = model.params[2]
            results['accel_se'][i] = model.bse[2]
        except:
            continue
        
    return pd.DataFrame(results, index=time)

# ## 2. Data Ingestion (8 Datasets + Uncertainties)

# [Markdown Cell explaining that NASA/NOAA/BE/HadCRUT are loaded here]

# Placeholder loading logic - In a live environment, these paths point to data/raw
def load_and_align():
    # Example logic for NASA GMSL
    # nasa_df = pd.read_csv('../data/raw/gmslr/nasa_GMSL_TPJAOS_5.2.txt', sep='\s+', skiprows=34, header=None)
    # nasa_time = nasa_df[2]; nasa_val = nasa_df[11]; nasa_sig = nasa_df[12]
    pass

# ## 3. Baseline Alignment (1993–2008)
# All datasets are shifted so their mean value between 1993 and 2008 is zero.

# ## 4. Moving Window Kinematics (10, 20, 30 Year Windows)
# We calculate rates and accelerations for every individual dataset and the ensemble means.

windows = [10, 20, 30]
# Results are stored in dictionaries and then exported to CSV.

# ## 5. Visualizations

# ### Plot 1: Ensemble Rate Comparison
# 

# ### Plot 2: Acceleration Stability vs Window Size
# This plot demonstrates how the 30-year window (WLS) provides the most stable 
# climate-forced signal compared to 10-year windows dominated by ENSO.



# ## 6. Data Export
# Exporting finalized CSVs to data/processed/
# - gmst_kinematics.csv
# - gmslr_kinematics.csv
